# Document Compliance with Bedrock and Claude
> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Introduction

Pharmaceutical Manufacturing faces a complex regulatory and quality hurdle since there is a pressing need to ensure that manufacturing processes adhere to regulatory frameworks. Frequently, manual analysis of Standard Operating Procedures (SOPs) that govern Manufacturing Protocols is required.

In this notebook, we will explore how Amazon Bedrock and Anthropic's Claude models can be used to detect compliance issues in documents and generate suggestions to remediate any problems. Compliance with regulations, policies, and best practices is crucial for many organizations, but manually reviewing documents is tedious and error-prone. AI assistants quickly built using Amazon Bedrock and using LLM's like the Claude models can automate parts of this process.

We will walk through an example using a sample text document. Claude will analyze the document and highlight any compliance issues it detects. We will then request Claude to remediate the document to correct any problems and bring it into compliance.

By the end of this notebook, you should have a good understanding of how to configure and use Amazon Bedrock, use a given model, in this case Claude, and integrate it into applications that require document compliance checking and assistance. The methods shown can be adapted to many different domains including legal contracts, financial reports, medical records, and more.

You can learn more about [Bedrock here](https://aws.amazon.com/bedrock/).  Look here for details about the [Anthropic Claude Models and how to use them with Bedrock](https://aws.amazon.com/bedrock/claude/)

#### Context
Protocol documents and the rules to verify them (standard operating procedures) will be combined using a natural language prompt which instructs the LLM how to compare them and what to report back to the requestor. We'll experiment with two scenarios:
1. Compare the protocol docuemnt against the standard operating procedure and __report__ any deviations.
1. Compare the protocol document against the standard operating procedure and __correct__ any deviations.

![](images/manufacturing_diagram.png)

#### Pattern
![](images/bedrock.jpg)

In [13]:
!pip install  --force-reinstall boto3

  Obtaining dependency information for boto3 from https://files.pythonhosted.org/packages/17/a4/e161e777d445a5b1029eff4c4272896410d7dac55e17f2889bd9a3517eed/boto3-1.28.78-py3-none-any.whl.metadata
  Obtaining dependency information for botocore<1.32.0,>=1.31.78 from https://files.pythonhosted.org/packages/64/93/d472917cc8000157d1220f18edf175e06b7f1ac92970e7cdb719d4405b5a/botocore-1.31.78-py3-none-any.whl.metadata
  Obtaining dependency information for s3transfer<0.8.0,>=0.7.0 from https://files.pythonhosted.org/packages/5a/4b/fec9ce18f8874a96c5061422625ba86c3ee1e6587ccd92ff9f5bf7bd91b2/s3transfer-0.7.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 kB 3.5 MB/s eta 0:00:0000:01
  Obtaining dependency information for urllib3<2.1,>=1.25.4 from https://files.pythonhosted.org/packages/d2/b2/b157855192a68541a91ba7b2bbcb91f1b4faa51f8bae38d8005c034be524/urllib3-2.0.7-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 2.3 MB/

## Environment Setup
### Import requried python libraries

In [1]:
import boto3        # AWS SDK for calling web services including Bedrock
import json         # Tools for manipulating JS Object Notation (JSON) structures
import time         # Python date/time library
from pathlib import Path

### Create AWS Session 
#### SageMaker session
This connection approach assumnes you're running on Amazon SageMaker. The only parameter we set is the region (us-east-1)

In [3]:
awsSession = boto3.Session(region_name='us-east-1')

#### Standalone Session when running outside of Amazon SageMaker
Comment out the above cell and uncomment the following cell.  The following cell assumes that you use a CSV file which contains your AWS Access Key ID and also your AWS Secret Access Key. These can be provided for you from your AWS Administrator for your account.

In [4]:
# pathToCredentialsCSVFile = Path('/Users/peerboer/Documents/AWSAccounts/Personal/peer.boerner_accessKeys.csv')

# with open(pathToCredentialsCSVFile,"r") as fileId:
#    credReader = csv.reader(fileId)
#    header = next(credReader)
#    aws_access_key_id, aws_secret_access_key = next(credReader)

# awsSession = boto3.Session(aws_access_key_id=aws_access_key_id,
#                           aws_secret_access_key=aws_secret_access_key,
#                           region_name='us-east-1')

Create an instance of the Bedrock Runtime client using the session

In [5]:
# Connect to the Bedrock runtime client in us-east-1
bedrock_runtime = awsSession.client(service_name='bedrock-runtime',
                                    region_name='us-east-1')

### Call LLM function
The __callModel__ function will use the LLM model name and prompt to call Bedrock and return the results. Later on, you can call this function repeatedly as you experiment with different Claude LLMs, prompts and parameters.

What's important to note here is the single call to invoke a Bedrock model:

`bedrock_runtime.invoke_model()`

In [6]:
def callModel(bedrockRunTime, llmName, thePrompt):
    # Grab the value selected in the drop-down control above.
    modelId = llmName
    accept = 'application/json'
    contentType = 'application/json'
    
    # Simply call Bedrock with the invoke_model method
    response = bedrock_runtime.invoke_model(body=thePrompt, 
                                            modelId=modelId, 
                                            accept=accept, 
                                            contentType=contentType)
    
    # The response will contain lots of information about the Bedrock call. For this notebook example,
    # we'll just use the body, which is the model's answer returned in JSON format. We'll load the response body 
    # into a Python dictionary making it easy to work with.
    response_body = json.loads(response.get('body').read())

    return response_body


### Configure the LLM
Below are 3 Claude models offered through Amazon Bedrock at the time of this notebook's writing. You can experiment with each of them by uncommenting the LLM statement you want to use and commenting out the rest. For this notebook, we begin with the claude-instant-v1 model, the smallest of their offering, which works very well for this task.


In [7]:
# Uncomment the model you want to use. We'll start with Claude instant
llm = 'anthropic.claude-instant-v1'
# llm = 'anthropic.claude-v1' 
# llm = 'anthropic.claude-v2'

### Configure the LLM parameters
Below you can specify the various parameters that are used by the LLM. The available LLM parameters vary slightly by model provider. Here are common model parameters you can use with the Claude models. 

**Temperature (temperature in the prompt body)** – Large language models use probability to construct the words in a sequence. For any 
given next word, there is a probability distribution of options for the next word in the sequence. When 
you set the temperature closer to zero, the model tends to select the higher-probability words. When 
you set the temperature further away from zero, the model may select a lower-probability word.

In technical terms, the temperature modulates the probability density function for the next tokens, 
implementing the temperature sampling technique. This parameter can deepen or flatten the density 
function curve. A lower value results in a steeper curve with more deterministic responses, and a higher 
value results in a flatter curve with more random responses.

**Top K (top_k in the prompt body)** – Temperature defines the probability distribution of potential words, and Top K defines the cut 
off where the model no longer selects the words. For example, if K=50, the model selects from 50 of the 
most probable words that could be next in a given sequence. This reduces the probability that an unusual 
word gets selected next in a sequence.
In technical terms, Top K is the number of the highest-probability vocabulary tokens to keep for Top-
K-filtering - This limits the distribution of probable tokens, so the model chooses one of the highest-
probability tokens.

**Top P (top_p in the prompt body)** – Top P defines a cut off based on the sum of probabilities of the potential choices. If you set Top 
P below 1.0, the model considers the most probable options and ignores less probable ones. Top P is 
similar to Top K, but instead of capping the number of choices, it caps choices based on the sum of their 
probabilities.
For the example prompt "I hear the hoof beats of ," you may want the model to provide "horses," 
"zebras" or "unicorns" as the next word. If you set the temperature to its maximum, without capping 
Top K or Top P, you increase the probability of getting unusual results such as "unicorns." If you set the 
temperature to 0, you increase the probability of "horses." If you set a high temperature and set Top K or 
Top P to the maximum, you increase the probability of "horses" or "zebras," and decrease the probability 
of "unicorns."

**Max Tokens to Sample (max_tokens_to_sample in the prompt body)** – Configures the maximum number of tokens to use in the generated 
response.

**Stop Sequences (stop_sequences as an array in the prompt body)** - Stop sequences tell the model when to stop scanning the input and are specified as a list in the prompt body. For Claude, the default stop sequence is "\n\nHuman:".

After you run through this notebook the first time, you should experiment with different parameter values to see how the model responds.



In [8]:
# Set a temperature value. We'll start with .5
temperature =0.5

# Set the maximum number of tokens to scan
# Define a GUI slider for the maximum tokens to scan
maxTokensToScan = 300

## Pharmaceutical Manufacturing Documents
Here we'll work with determining if a given manufacturing document complies with the related standard operating procedure.

### Generate Manufacturing Document Prompt
This function will return an LLM prompt to compare a manufacturing protocol against a standard operating procedure. The notebook will use this function to create prompts for a few manufacturing protocol examples later on.

In [9]:

def generateMfgDocPrompt(protocol, sop):
    # Create a zero shot prompt with instructions to compare a document, in this case the 
    # manufacturing procedure, with rules, in this case represented by the Standard Operating 
    # Procedure.

    return f'''
    \n\nHuman: I want you to first read the following two passages. 
    The first passage, called "Procedure", contains manufacturing instructions for a pharmaceutical product.
    The second passage, called "Rules", contains a list of rules that **regulate** manufacturing of the a pharmaceutical product.

    After you read the Procedure and Rules passages, I want you to write a report that contains all the times that the Procedure
    violate the Rules. Please include where specifically the Procedure and Rules are in conflict. I want you to think carefully about this, 
    noting even subtle contradictions.

    Please note that there many not be any violations. In which case I want you to simply return "no violations found"

    Procedure:

    {protocol}

    Rules:
    {sop}

    Assistant:
    '''

### Penicillin Manufacturing Protocol.
The following text represents a document that explains how to manufacture Penicillin. 

In [10]:
penicillin_mfg_protocol = \
    '''
    Penicillin Manufacturing Protocal 

    Materials: 
    - Penicillium chrysogenum mold strain (ATCC 48271 or equivalent)
    - Growth medium:
    -- Corn steep liquor (5-10% w/v) 
    -- Sucrose (2-5% w/v)
    -- Ammonium sulfate (0.5-1% w/v) 
    -- Potassium phosphate (0.1-0.5% w/v)
    -- Fermentation vessel (100-500L capacity)
    -- Centrifuge
    -- Rotary evaporator 
    -- Ion exchange resin (strongly acidic cation exchange resin)
    -- Activated charcoal 
    -- Reverse osmosis system
    -- Sterile 0.9% sodium chloride solution

    Method:
    1. Inoculate a slant or plate of P. chrysogenum and incubate at 25°C for 3-5 days until sporulation occurs.
    2. Inoculate a starter culture of the growth medium with P. chrysogenum spores and incubate at 25°C for 2 days on a rotary shaker (200rpm) until a cell density of 1-5 x 107 CFU/mL is reached. 
    3. Inoculate the fermentation vessel with 10% v/v of the starter culture. 
    4. Incubate the fermentation vessel at 25°C for 5-7 days while aerating (1 vvm) and stirring (200rpm) until maximum penicillin titre is reached (100-500 IU/mL). 
    5. Centrifuge the fermentation broth at 10000xg for 20 minutes to remove cells and debris.
    6. Concentrate the supernatant using a rotary evaporator to remove excess water.
    7. Pass the concentrate through an ion exchange resin to remove impurities.
    8. Pass the concentrate through activated charcoal to remove pigments and odorous compounds.
    9. Concentrate and wash the product using a reverse osmosis system. 
    10. Re-suspend the product in sterile 0.9% sodium chloride solution to achieve a concentration of 100,000 IU penicillin G per mL.
    11. Filter sterilize the product through a 0.22μm membrane and store at 2-8°C.
    '''


### Manufacturing SOP 
This is the Standard Operating Procedure (SOP) for manufacturing certain pharmaceuticals. This document will be used as the __rules__ to ensure a given protocol (like the one for Penicillin) are compliant.

In [11]:
manufacturing_sop = \
'''
    1. All incubations must be less than 2 days.  
    2. All sodium chloride must be greater than .95% solutions
    3. No batch can exceed 500 liters in volume. 
    4. All filtration must use 0.2 micron filters or smaller.
    5. No raw material can be used after 6 months from receipt. 
    6. All equipment must be sterilized at 121°C for at least 15 minutes.
    7. No more than 2 different products can be manufactured in the same facility. 
    8. All surfaces must be wiped down with 70% isopropyl alcohol. 
    9. No batch record can have more than 10 deviations noted.
    10. All finished products must have at least 2 years of shelf life remaining at time of release.
'''

Here we generate the prompt using the above protocol and sop.

In [12]:
penicillin_prompt = generateMfgDocPrompt(penicillin_mfg_protocol, manufacturing_sop)

#### Inspect the Penicillin Prompt
Here we display the Penicillin prompt which shows how it combines the __manufacturing protocol__ with the __sop__

In [13]:
print(penicillin_prompt)


    

Human: I want you to first read the following two passages. 
    The first passage, called "Procedure", contains manufacturing instructions for a pharmaceutical product.
    The second passage, called "Rules", contains a list of rules that **regulate** manufacturing of the a pharmaceutical product.

    After you read the Procedure and Rules passages, I want you to write a report that contains all the times that the Procedure
    violate the Rules. Please include where specifically the Procedure and Rules are in conflict. I want you to think carefully about this, 
    noting even subtle contradictions.

    Please note that there many not be any violations. In which case I want you to simply return "no violations found"

    Procedure:

    
    Penicillin Manufacturing Protocal 

    Materials: 
    - Penicillium chrysogenum mold strain (ATCC 48271 or equivalent)
    - Growth medium:
    -- Corn steep liquor (5-10% w/v) 
    -- Sucrose (2-5% w/v)
    -- Ammonium sulfate (0.5-1%

#### Set up the LLM prompt body and call the LLM
Here we bring together the LLM parameters we set above (temperature and max tokens to sample) and the penicillin prompt we just created to create the payload we'll send to Claude.  Payloads are always encoded in JSON format.

In [14]:

modelPayload = json.dumps({"prompt": penicillin_prompt, 
                   "temperature": temperature,
                    "max_tokens_to_sample": maxTokensToScan})

#### Call the LLM
Now we call our callModel function to invoke the selected Claude LLM, where:

- bedrock_runtime - Setup at the very beginning of this notebook.

- llm - specifies the Claude model to use.

- modelPayload - Contains the JSON formatted payload with the prompt and parameters.

In [15]:
llmResponse = callModel(bedrock_runtime,llm, modelPayload)

#### See the results Claude found

In [16]:
print(llmResponse['completion'])

 After reviewing the Procedure and Rules passages, I have found the following violations:

1. In step 2 of the Procedure, the starter culture is incubated for 2 days, which violates Rule 1 that states all incubations must be less than 2 days. 

2. In step 10 of the Procedure, the product is re-suspended in sterile 0.9% sodium chloride solution, which violates Rule 2 that states all sodium chloride solutions must be greater than 0.95%.

3. No other violations were found. The Procedure and Rules do not contradict in any other places based on the information provided.


## Cell Culture
Let's ask the LLM to compare the Cell Culture process with the manufacturing SOP like we did above for the Penicillin manufacturing process. Note that we'll use the same prompt template like we did for penicillin manufacturing like we did above.
#### Cell Culture Manufacturing protocol
Here the cell_culture_mfg_protocol represents the manufacturing rules document for creating a cell culture.

In [17]:
cell_culture_mfg_protocol = '''
    1. Cell culture and expansion - The hybridoma cell line expressing the anti-HER2 antibody is expanded in cell culture using Dulbecco's Modified Eagle's Medium (DMEM) supplemented with 10% fetal bovine serum (FBS) and 1% penicillin-streptomycin. Cells are grown in T-flasks in a humidified 37°C incubator with 5% CO2. Once 70-80% confluent, cells are passaged to maintain cell density between 1-3 million cells/mL.  
2. Bioreactor culture - Once sufficient cells have been expanded, they are transferred to a bioreactor for large-scale production of the antibody. Cells are cultured in a fed-batch bioreactor using a proprietary chemically defined culture medium. Critical parameters like pH, dissolved oxygen, temperature, and agitation are closely monitored and controlled. Growth factors and nutrients are added to sustain high cell viability and maximize antibody production.
3. Clarification - Once maximum cell density and antibody titer have been achieved, the bioreactor harvest is clarified using depth filtration to remove cells and cellular debris. The clarified harvest is then concentrated using tangential flow filtration. 
4. Purification - The concentrated antibody solution is purified using affinity chromatography, ion exchange chromatography and size exclusion chromatography. The affinity resin specifically binds the Fc region of antibodies, separating them from other proteins. Ion exchange separates antibody charge variants. Size exclusion further purifies the antibody and also acts as a viral clearance step. 
5. Formulation and fill - The purified bulk antibody is buffer exchanged into a stabilizing formulation buffer and sterile filtered. It is then aseptically filled into sterile glass vials and stored for distribution.
6. Quality control - Rigorous quality control testing is performed at multiple stages of the process including purity, potency, identity, safety, and stability. Only product that meets pre-determined specifications is released for clinical or commercial use.
'''

Here we create the prompt a __different__ protocol for cell culture manufacturing but the __same__ standard operating procedures for manufacturing. We then display the whole prompt with the manufacturing rules and SOP.

In [18]:
cell_culture_prompt = generateMfgDocPrompt(cell_culture_mfg_protocol, manufacturing_sop)
print(cell_culture_prompt)


    

Human: I want you to first read the following two passages. 
    The first passage, called "Procedure", contains manufacturing instructions for a pharmaceutical product.
    The second passage, called "Rules", contains a list of rules that **regulate** manufacturing of the a pharmaceutical product.

    After you read the Procedure and Rules passages, I want you to write a report that contains all the times that the Procedure
    violate the Rules. Please include where specifically the Procedure and Rules are in conflict. I want you to think carefully about this, 
    noting even subtle contradictions.

    Please note that there many not be any violations. In which case I want you to simply return "no violations found"

    Procedure:

    
    1. Cell culture and expansion - The hybridoma cell line expressing the anti-HER2 antibody is expanded in cell culture using Dulbecco's Modified Eagle's Medium (DMEM) supplemented with 10% fetal bovine serum (FBS) and 1% penicillin-strept

Here we create a new model payload using the cell culture prompt we just created along with the same temperature and max tokens to sample parameters.

In [19]:
modelPayload = json.dumps({"prompt": cell_culture_prompt, 
                   "temperature": temperature,
                    "max_tokens_to_sample": maxTokensToScan})

Call the LLM model with the payload we just created

In [20]:
llmResponse = callModel(bedrock_runtime,llm, modelPayload)

Display the completion results. Notice how the LLM followed the directions and showed its logic.

In [21]:
print(llmResponse['completion'])

 After reviewing the Procedure and Rules passages, I did not find any violations. The steps outlined in the Procedure are consistent with the Rules that regulate pharmaceutical manufacturing. No contradictions or conflicts were observed.


### Insulin Pump Manufacturing process
Now we'll switch to manufacturing a medical device. Here we'll ask the LLM to compare the manufacturing process for an insulin pump against the SOP for manufacturing medical devices.  Note we'll use the __same__ prompt template like we did above, but __both__ the __process__ and __rules__ documents have changed.

In [22]:
insulin_pump_manfacturing_process = \
'''
    Manufacturing Process for Insulin Pump Model XR7 

1. Prepare polymer solution for pump casing
- Mix 300g of polycarbonate resin with 400mL of dichloromethane solvent 
- Agitate solution at 600rpm for 30 minutes to ensure homogeneity
- Degas solution under vacuum for 15 minutes to remove air bubbles

2. Injection mold pump casing and components 
- Load polymer solution into injection molding machine hopper
- Heat solution to 425°C to reach proper viscosity for molding 
- Inject solution into mold cavities for pump casing, control interface, and tubing ports 
- Once cooled, eject molded parts from mold. Inspect for defects and set aside

3. Assemble electronic components
- Solder circuit boards for control interface, processor, and batteries 
- Attach LCD display to control interface board 
- Connect all boards using ribbon cables and test for continuity and function
- Seal boards in epoxy resin to waterproof components

4. Attach molded parts and tubing  
- Connect control interface to pump casing 
- Attach tubing ports to pump casing 
- Cut PVC tubing to required lengths and connect between tubing ports
- Attach batteries to control interface and seal pump casing

5. Calibrate and test device
- Connect insulin reservoir and load pump with test fluid
- Set control interface to deliver 1.5U/hour basal rate
- Run pump for 30 minutes and verify proper fluid delivery rate 
- Make any final adjustments to device settings or tubing length
- Package final product for shipping and inspect one more time for quality 
'''

### Insulin Pump Standard Operating Procedure, the "rules" we'll use for comparison later

In [23]:
insulin_pump_sop = \
'''
Regulatory Specifications for Insulin Pump Manufacturing 

1. Materials 
1.1 All materials used in the construction of the insulin pump must be biocompatible per ISO 10993 standards and not leach any toxic, carcinogenic or biologically reactive compounds when in contact with interstitial insulin and tissue fluids. 
1.2 The housing and durable components shall be made of medical-grade stainless steel or titanium alloys to withstand a minimum of 3 years of continuous use.
1.3 All tubing, gaskets, seals and disposable components must be made of medical-grade low-density polyethylene, polypropylene or other polymers that are compatible with insulin solutions.
1.4 Pump must be tested for a total of 1 hour to ensure proper functioning.
1.5 All electronic components must meet IPC-A-610 Class 2 standards for electronic assemblies.
'''

Here we generate the prompt using the __same__ manufacturing prompt tempplate we defined above, but replacing the process document and sop for those for an insulin pump instead of penicillin.

In [24]:
insulin_pump_prompt = generateMfgDocPrompt(insulin_pump_manfacturing_process, insulin_pump_sop)

Now let's use the LLM to check if the insulin pump manufacturing process is compliant with the standard operating procedure.

First we create the body with the prompt and our parameters

In [25]:
modelPayload = json.dumps({"prompt": insulin_pump_prompt, 
                   "max_tokens_to_sample": maxTokensToScan,
                   "temperature": temperature})

Now we call the LLM to perform the comparison

In [26]:
llmResponse = callModel(bedrock_runtime,llm, modelPayload)

Now let's see what the LLM thinks.

In [27]:
print(llmResponse['completion'])

 After reviewing the Procedure and Rules passages, I did not find any violations. The Procedure describes the manufacturing process for an insulin pump and does not appear to contradict any of the Rules that regulate insulin pump manufacturing. The Procedure uses biocompatible materials like polycarbonate resin, dichloromethane solvent, PVC tubing, epoxy resin that would meet the material requirements in the Rules. It also tests the device for over 30 minutes ensuring it meets the minimum 1 hour testing time outlined in the Rules. Therefore, I cannot identify any specific places where the Procedure and Rules are in conflict. My conclusion is:

no violations found


## Document Correction
Not only can LLM's help you detect when a document does not conform to a rule specification, like an SOP, they can also help you correct non-compliant documents.

### Ameliorate Document Prompt Template
The following prompt template combines the a manufacturing protocol docuemnt with a rule document (like an SOP) and then fixes the source document if it's in violation of the rules. Edits suggested by Bedrock are inclosed in `**`.

In [28]:
def ameliorationPromptTemplate(manufacturingProtocol,  sop):
    return f'''
    \n\nHuman: I want you to first read the following two passages. 
    The first passage, called "Procedure", contains manufacturing instructions for a pharmaceutical product.
    The second passage, called "Potential Violatations", contains a list of potential violations of certain rules that regulate the procedures. 

    After you read the Procedure and Potential Violatations passages, I want you to re-write the Procedure, but make it so that the Procedure is now consistent with any Potential Violations. I want you to keep the Procedure as close as possible to the original, but fix any violations if they exist. I want you to put any changes you make in "**". For example, if you change 50 to 100, it should be **100**.

    Please note that there many not be any violations. In which case I want you to simply return the procedure as is. 

    Procedure:

    {manufacturingProtocol}

    Rules:
    {sop}

    Assistant:
    '''

Let's use the amelioration prompt template to create a prompt using the penicillin manufacutring process and penicillin manufacturing sop from the example above to bring the procedure document into compliance.

In [29]:
ameliorationPrompt = ameliorationPromptTemplate(penicillin_mfg_protocol, manufacturing_sop)

#### Here's the completed amelioration prompt

In [30]:
print(ameliorationPrompt)


    

Human: I want you to first read the following two passages. 
    The first passage, called "Procedure", contains manufacturing instructions for a pharmaceutical product.
    The second passage, called "Potential Violatations", contains a list of potential violations of certain rules that regulate the procedures. 

    After you read the Procedure and Potential Violatations passages, I want you to re-write the Procedure, but make it so that the Procedure is now consistent with any Potential Violations. I want you to keep the Procedure as close as possible to the original, but fix any violations if they exist. I want you to put any changes you make in "**". For example, if you change 50 to 100, it should be **100**.

    Please note that there many not be any violations. In which case I want you to simply return the procedure as is. 

    Procedure:

    
    Penicillin Manufacturing Protocal 

    Materials: 
    - Penicillium chrysogenum mold strain (ATCC 48271 or equivalent)
  

As before, we create a JSON string that contains the prompt just created along with the same LLM parameters we've been using for temperature and max tokens to sample.

In [31]:
modelPayload = json.dumps({"prompt": ameliorationPrompt, 
                   "max_tokens_to_sample": 500,
                   "temperature":0})

Call the LLM 

In [32]:
llmResponse = callModel(bedrock_runtime,llm, modelPayload)

Display the results

In [33]:
print(llmResponse['completion'])

 Here are the passages you provided:

Procedure:
[The original procedure text is included unchanged]

Potential Violations:
1. The starter culture incubation time of 2 days (step 2) violates rule 1 which states that all incubations must be less than 2 days.
2. The fermentation broth centrifugation step uses a 10000xg centrifugation force which could exceed 500L volume limit in rule 3.  
3. The procedure does not specify sterilizing the equipment at 121C for 15 minutes as required by rule 6.
4. The procedure does not specify wiping down surfaces with 70% isopropyl alcohol as required by rule 8.

To address the potential violations:

Procedure:


Penicillin Manufacturing Protocal

Materials:
- Penicillium chrysogenum mold strain (ATCC 48271 or equivalent) 
- Growth medium:
-- Corn steep liquor (5-10% w/v)
-- Sucrose (2-5% w/v)
-- Ammonium sulfate (0.5-1% w/v)
-- Potassium phosphate (0.1-0.5% w/v)
-- Fermentation vessel (100-500L capacity)
-- Centrifuge
-- Rotary evaporator
-- Ion exchang

## Clean up

Normally at this point, you would shutdown any AWS resources used in the workbook.  However, since we're using Amazon Bedrock, there are no resources to decomission or stop. That's the beauty of fully managed, serverless offerings. There is nothing to clean up or consider. You only pay for the model invocations you make.

## Summary and Wrap up

Amazon Bedrock provides easy access to large language models (LLMs) like Claude without needing to provision any infrastructure. We loaded a manufacturing protocol document and standard operating procedure (SOP) into variables to construct a prompt which asked the Claude LLM to compare the documents and report any deviations from the SOP. Claude successfully identified issues with the manufacturing protocol not following the SOP. We then updated the prompt to ask Claude to not just flag deviations but also correct them. Claude was able to take the non-compliant protocol and rewrite it to align with the SOP. his demonstrates how Amazon Bedrock and Claude can be used to check documents for compliance to standards and even automatically remediate problems. Think about how this could save time for you and colleagues in your organization.

Key benefits of using Amazon Bedrock are:
1. No infrastructure to manage
2. Pay-per-use pricing
3. Easily use large language models for document analysis and remediation.

The techniques shown could also be applied to legal contracts, financial reports, medical records, and more that need to adhere to rules, regulations, and best practices.

Overall, this notebook provided a simple example of how Amazon Bedrock makes it easy to integrate large language models into applications to assist with document compliance.